In [1]:
import gym
import numpy as np
import sys
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
from google.colab import files

In [2]:
!pip install kaggle_environments

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.1/134.1 kB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_c

In [3]:
from kaggle_environments import evaluate, make, utils
env = make("connectx", debug=True)
agent1 = 'random'
agent2 = 'negamax'

env.run([agent1, agent2])
env.render(mode="ipython", width=600, height=500, header=False)

DQN

In [4]:
def random_episode_DQN(policy):
    env.reset()
    original_stdout = sys.stdout
    r1,r2 = 0,0
    current_state = tuple(np.zeros(42))
    states, actions, rewards = [current_state], [], []
    while r1 == 0 and r2 != None:
      if tuple(current_state) in policy:
        a1 = policy[tuple(current_state)]
      else:
        a1 = np.random.randint(0,7)
      a2 = np.random.randint(0,7)
      try:
        sys.stdout = open('/dev/null', 'w')
        e1 = env.step([a1,a2])
        sys.stdout = original_stdout
      except:
        r1,r2 = -0.5,-0.5
        break
      r1 = e1[0]['reward']
      r2 = e1[1]['reward']
      if r1 == 0 and r2!= None:
        sys.stdout = open('/dev/null', 'w')
        e = env.step([a1,a2])
        sys.stdout = original_stdout
        r1,r2 = e[0]['reward'],e[1]['reward']
        current_state = e[0]['observation']['board']
      if (r1 == None) or (r2 == None):
        r1,r2 = -0.5,-0.5
      else:
        r2 = -1*r1
        current_state = e1[0]['observation']['board']
      states.append(current_state)
      rewards.append(r1)
      actions.append(a1)
    sys.stdout = original_stdout
    return r1,r2, states, rewards, actions

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
import numpy as np

class Connect4:
    def __init__(self):
        self.rows = 6
        self.cols = 7
        self.board = np.zeros((self.rows, self.cols))
        self.player = 1
        self.winning_length = 4

    def reset(self):
        self.board = np.zeros((self.rows, self.cols))
        self.player = 1
        return self.board.copy()

    def is_valid_move(self, col):
        return self.board[0][col] == 0

    def make_move(self, col):
        for row in range(self.rows - 1, -1, -1):
            if self.board[row][col] == 0:
                self.board[row][col] = self.player
                return row, col

    def check_winner(self, row, col):
        directions = [(0, 1), (1, 0), (1, 1), (1, -1)]
        for dr, dc in directions:
            count = 1
            r, c = row + dr, col + dc
            while 0 <= r < self.rows and 0 <= c < self.cols and self.board[r][c] == self.player:
                count += 1
                r += dr
                c += dc
            r, c = row - dr, col - dc
            while 0 <= r < self.rows and 0 <= c < self.cols and self.board[r][c] == self.player:
                count += 1
                r -= dr
                c -= dc
            if count >= self.winning_length:
                return True
        return False

    def is_board_full(self):
        return np.all(self.board != 0)

    def step(self, action):
        if self.player == 1:
          win = 1
        else:
          win = -1
        if not self.is_valid_move(action):
            return self.board.copy(), -0.8*win, True, {}
        row, col = self.make_move(action)
        if self.check_winner(row, col):
            return self.board.copy(), win, True, {}
        if self.is_board_full():
            return self.board.copy(), 0, True, {}
        self.player = 3 - self.player  # Switch player
        return self.board.copy(), 0, False, {}

    def render(self):
        print(self.board)

    @property
    def observation_space(self):
        return self.board.shape

    @property
    def action_space(self):
        return self.cols


In [ ]:
from re import VERBOSE
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from collections import deque
import random
from tqdm import tqdm

# Define the Connect4 environment and other constants
# Implement the environment-specific functions: reset(), step(), etc.

# Define the DQN agent class
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)  # Replay memory
        self.gamma = 0.95  # Discount factor
        self.epsilon = 0.2  # Exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.999
        self.batch_size = 32
        self.online_network = self.build_network()
        self.target_network = self.build_network()
        self.update_target_network()

    def build_network(self):
        model = models.Sequential([
            layers.Input(shape=self.state_size),
            layers.Flatten(),
            layers.Dense(128, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(self.action_size)  # Output layer for Q-values
        ])
        model.compile(loss='mean_squared_error', optimizer=optimizers.Adam())
        return model

    def update_target_network(self):
        self.target_network.set_weights(self.online_network.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        q_values = self.online_network.predict(state, verbose=0)
        return np.argmax(q_values[0])

    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        minibatch = random.sample(self.memory, self.batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.online_network.predict(state, verbose=0)
            if done:
                target[0][action] = reward
            else:
                target[0][action] = reward + self.gamma * np.amax(self.target_network.predict(next_state, verbose=0)[0])
            self.online_network.fit(state, target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

class RandomPlayer:
    def __init__(self, action_space):
        self.action_space = action_space

    def choose_action(self, state):
        valid_actions = [action for action in range(self.action_space) if env.is_valid_move(action)]
        return random.choice(valid_actions) if valid_actions else None

def WDL_string(WDL):
  return str(WDL[1]) + ' | ' + str(WDL[0]) + ' | ' + str(WDL[-1])

# Initialize the Connect4 environment and the agent
env = Connect4()
state_size = env.observation_space
action_size = env.action_space
agent = DQNAgent(state_size, action_size)
random_model = RandomPlayer(env.action_space)
WDL = {1:0, 0:0, -1:0, -0.8: 0, 0.8: 0}
flag = 0

# Train the agent
for episode in range(1000):
    state = env.reset()
    state = np.expand_dims(state, axis = 0)
    states = []
    actions = []
    rewards = []
    for time_step in range(50):
        # if time_step == 43:
        #   flag=1
        #   break
        if time_step%2 == 0:
          action = agent.act(state)
        else:
          action = random_model.choose_action(state)
        # states.append(state)
        # actions.append(action)
        next_state, reward, done, _ = env.step(action)
        next_state = np.expand_dims(next_state, axis = 0)
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            WDL[reward] += 1
            rewards.append(reward)
            print("Episode: {}/{}, Steps: {}, Reward: {}, WDL: {}".format(episode, 1000, time_step, reward, WDL_string(WDL)))
            break
    if episode % 10 == 0:
        agent.update_target_network()
    agent.replay()
    # if flag==1:
    #   break

# Evaluate the agent
# Implement code to play Connect4 using the trained agent


Episode: 0/1000, Steps: 8, Reward: 1, WDL: 1 | 0 | 0
Episode: 1/1000, Steps: 6, Reward: 1, WDL: 2 | 0 | 0
Episode: 2/1000, Steps: 14, Reward: 1, WDL: 3 | 0 | 0
Episode: 3/1000, Steps: 6, Reward: 1, WDL: 4 | 0 | 0
Episode: 4/1000, Steps: 10, Reward: 1, WDL: 5 | 0 | 0
Episode: 5/1000, Steps: 13, Reward: -1, WDL: 5 | 0 | 1
Episode: 6/1000, Steps: 12, Reward: 1, WDL: 6 | 0 | 1
Episode: 7/1000, Steps: 18, Reward: 1, WDL: 7 | 0 | 1
Episode: 8/1000, Steps: 13, Reward: -1, WDL: 7 | 0 | 2
Episode: 9/1000, Steps: 24, Reward: -0.8, WDL: 7 | 0 | 2
Episode: 10/1000, Steps: 17, Reward: -1, WDL: 7 | 0 | 3
Episode: 11/1000, Steps: 10, Reward: -0.8, WDL: 7 | 0 | 3
Episode: 12/1000, Steps: 12, Reward: 1, WDL: 8 | 0 | 3
Episode: 13/1000, Steps: 10, Reward: 1, WDL: 9 | 0 | 3
Episode: 14/1000, Steps: 20, Reward: -0.8, WDL: 9 | 0 | 3
Episode: 15/1000, Steps: 16, Reward: 1, WDL: 10 | 0 | 3
Episode: 16/1000, Steps: 8, Reward: 1, WDL: 11 | 0 | 3
Episode: 17/1000, Steps: 16, Reward: -0.8, WDL: 11 | 0 | 3
Episod

In [7]:
from re import VERBOSE
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from collections import deque
import random
from tqdm import tqdm

# Define the Connect4 environment and other constants
# Implement the environment-specific functions: reset(), step(), etc.

# Define the DQN agent class
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)  # Replay memory
        self.gamma = 0.95  # Discount factor
        self.epsilon = 0.2  # Exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.999
        self.batch_size = 32
        self.online_network = self.build_network()
        self.target_network = self.build_network()
        self.update_target_network()

    def build_network(self):
        model = models.Sequential([
            layers.Input(shape=self.state_size),
            layers.Flatten(),
            layers.Dense(128, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(self.action_size)  # Output layer for Q-values
        ])
        model.compile(loss='mean_squared_error', optimizer=optimizers.Adam())
        return model

    def update_target_network(self):
        self.target_network.set_weights(self.online_network.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        q_values = self.online_network.predict(state, verbose=0)
        return np.argmax(q_values[0])

    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        minibatch = random.sample(self.memory, self.batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.online_network.predict(state, verbose=0)
            if done:
                target[0][action] = reward
            else:
                target[0][action] = reward + self.gamma * np.amax(self.target_network.predict(next_state, verbose=0)[0])
            self.online_network.fit(state, target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

class RandomPlayer:
    def __init__(self, action_space):
        self.action_space = action_space

    def choose_action(self, state):
        valid_actions = [action for action in range(self.action_space) if env.is_valid_move(action)]
        return random.choice(valid_actions) if valid_actions else None

def WDL_string(WDL):
  return str(WDL[1]) + ' | ' + str(WDL[0]) + ' | ' + str(WDL[-1])

# Initialize the Connect4 environment and the agent
env = Connect4()
state_size = env.observation_space
action_size = env.action_space
agent = DQNAgent(state_size, action_size)
random_model = RandomPlayer(env.action_space)
WDL = {1:0, 0:0, -1:0, -0.8: 0, 0.8: 0}
flag = 0

# Train the agent
for episode in range(1000):
    state = env.reset()
    state = np.expand_dims(state, axis = 0)
    states = []
    actions = []
    rewards = []
    for time_step in range(50):
        # if time_step == 43:
        #   flag=1
        #   break
        if time_step%2 == 0:
          action = agent.act(state)
        else:
          action = random_model.choose_action(state)
        # states.append(state)
        # actions.append(action)
        next_state, reward, done, _ = env.step(action)
        next_state = np.expand_dims(next_state, axis = 0)
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            WDL[reward] += 1
            rewards.append(reward)
            print("Episode: {}/{}, Steps: {}, Reward: {}, WDL: {}".format(episode, 1000, time_step, reward, WDL_string(WDL)))
            break
    if episode % 10 == 0:
        agent.update_target_network()
    agent.replay()
    # if flag==1:
    #   break

# Evaluate the agent
# Implement code to play Connect4 using the trained agent


Episode: 0/1000, Steps: 10, Reward: 1, WDL: 1 | 0 | 0
Episode: 1/1000, Steps: 9, Reward: -1, WDL: 1 | 0 | 1
Episode: 2/1000, Steps: 12, Reward: 1, WDL: 2 | 0 | 1
Episode: 3/1000, Steps: 10, Reward: 1, WDL: 3 | 0 | 1
Episode: 4/1000, Steps: 24, Reward: 1, WDL: 4 | 0 | 1
Episode: 5/1000, Steps: 20, Reward: -0.8, WDL: 4 | 0 | 1
Episode: 6/1000, Steps: 8, Reward: 1, WDL: 5 | 0 | 1
Episode: 7/1000, Steps: 22, Reward: -0.8, WDL: 5 | 0 | 1
Episode: 8/1000, Steps: 14, Reward: 1, WDL: 6 | 0 | 1
Episode: 9/1000, Steps: 20, Reward: -0.8, WDL: 6 | 0 | 1
Episode: 10/1000, Steps: 12, Reward: 1, WDL: 7 | 0 | 1
Episode: 11/1000, Steps: 16, Reward: 1, WDL: 8 | 0 | 1
Episode: 12/1000, Steps: 14, Reward: 1, WDL: 9 | 0 | 1
Episode: 13/1000, Steps: 18, Reward: 1, WDL: 10 | 0 | 1
Episode: 14/1000, Steps: 14, Reward: 1, WDL: 11 | 0 | 1
Episode: 15/1000, Steps: 18, Reward: 1, WDL: 12 | 0 | 1
Episode: 16/1000, Steps: 12, Reward: 1, WDL: 13 | 0 | 1
Episode: 17/1000, Steps: 10, Reward: 1, WDL: 14 | 0 | 1
Episode

In [8]:
model = agent

In [9]:
import pickle

# Assuming your model is named 'model'
# Save the model to a file
with open('DQN_FCNN.pkl', 'wb') as f:
    pickle.dump(model, f)

# Load the model from the file
with open('DQN_FCNN.pkl', 'rb') as f:
    loaded_model = pickle.load(f)


In [10]:
from google.colab import files
files.download('DQN_FCNN.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#agent = loaded_model

In [11]:
# Define the random model (opponent)
import random

class RandomPlayer:
    def __init__(self, action_space):
        self.action_space = action_space

    def choose_action(self, state):
        valid_actions = [action for action in range(self.action_space) if env.is_valid_move(action)]
        return random.choice(valid_actions) if valid_actions else None

# Instantiate the random model
random_model = RandomPlayer(env.action_space)

# Define evaluation parameters
num_games = 100
win_count_dqn = 0
win_count_random = 0
draw_count = 0

# Simulate games
for _ in range(num_games):
    state = env.reset()
    done = False
    while not done:
        # DQN agent's turn
        action_dqn = agent.act(np.expand_dims(state, axis=0))
        next_state, reward, done, _ = env.step(action_dqn)
        if done:
            if reward == 1:
                win_count_dqn += 1
            elif reward == -1:
                win_count_random += 1
            else:
                draw_count += 1
            break

        # Random model's turn
        action_random = random_model.choose_action(state)
        next_state, reward, done, _ = env.step(action_random)
        if done:
            if reward == 1:
                win_count_random += 1
            elif reward == -1:
                win_count_dqn += 1
            else:
                draw_count += 1
            break

        state = next_state

# Calculate win rates
win_rate_dqn = win_count_dqn / num_games
win_rate_random = win_count_random / num_games
draw_rate = draw_count / num_games

print("DQN Agent Win Rate:", win_rate_dqn)
print("Random Model Win Rate:", win_rate_random)
print("Draw Rate:", draw_rate)


DQN Agent Win Rate: 0.92
Random Model Win Rate: 0.0
Draw Rate: 0.08
